In [1]:
#export
__all__ = ["grep", "grepTemplate"]
import re, k1lib
from k1lib.cli.init import BaseCli, Table, Row; import k1lib.cli as cli
from collections import deque; from typing import Iterator

In [23]:
#export
inf = float("inf")
class grep(BaseCli):
    def __init__(self, pattern:str, before:int=0, after:int=0, N:int=float("inf"), sep:bool=False, col:int=None):
        """Find lines that has the specified pattern.
Example::

    # returns ['d', 'd']
    "abcde12d34" | grep("d") | deref()
    # returns ['c', 'd', '2', 'd'], 2 sections of ['c', 'd'] and ['2', 'd']
    "abcde12d34" | grep("d", 1) | deref()
    # returns ['c', 'd']
    "abcde12d34" | grep("d", 1, N=1) | deref()
    # returns ['d', 'e', 'd', '3', '4'], 2 sections of ['d', 'e'] and ['d', '3', '4']
    "abcde12d34" | grep("d", 0, 3).till("e") | deref()
    # returns [['0', '1', '2'], ['3', '1', '4']]
    "0123145" | grep("1", 2, 1, sep=True) | deref()

You can also separate out the sections::

    # returns [['c', 'd'], ['2', 'd']]
    "abcde12d34" | grep("d", 1, sep=True) | deref()
    # returns [['c', 'd']]
    "abcde12d34" | grep("d", 1, N=1, sep=True) | deref()
    # returns [['1', '2', '3'], ['1', '4', '5']]
    "0123145" | grep("1", sep=True).till() | deref()

See also: :class:`~k1lib.cli.structural.groupBy`

:param pattern: regex pattern to search for in a line
:param before: lines before the hit. Outputs independent lines
:param after: lines after the hit. Outputs independent lines
:param N: max sections to output
:param sep: whether to separate out the sections as lists
:param col: searches for pattern in a specific column"""
        super().__init__()
        self.pattern = re.compile(pattern)
        self.before = before; self.after = after; self.col = col
        self.N = N; self.sep = sep; self.tillPattern = None
    def till(self, pattern:str=None):
        """Greps until some other pattern appear. Inclusive, so you might want to
trim the last line. Example::

    # returns ['5', '6', '7', '8'], includes last item
    range(10) | join("") | grep("5").till("8") | deref()
    # returns ['d', 'e', 'd', '3', '4']
    "abcde12d34" | grep("d").till("e") | deref()
    # returns ['d', 'e']
    "abcde12d34" | grep("d", N=1).till("e") | deref()

If initial pattern and till pattern are the same, then you don't have use this method at
all. Instead, do something like this::

    # returns ['1', '2', '3']
    "0123145" | grep("1", after=1e9, N=1) | deref()"""
        if pattern == self.pattern.pattern: pattern = None
        # "\ue000" is in unicode's private use area, so extremely unlikely that we
        # will actually run into it in normal text processing, because it's not text
        self.tillPattern = re.compile(pattern or "\ue000")
        self.tillAfter = self.after; self.after = inf; return self
    def __ror__(self, it:Iterator[str]) -> Iterator[str]:
        self.sectionIdx = 0; tillPattern = self.tillPattern; col = self.col
        if self.sep:
            self.sep = False; elems = []; idx = 0
            for line in (it | self):
                if self.sectionIdx > idx: # outputs whatever remaining
                    if len(elems) > 0: yield list(elems)
                    idx = self.sectionIdx; elems = []
                elems.append(line)
            yield list(elems)
            self.sep = True; return
        queue = deque([], self.before); counter = 0 # remaining lines after to display
        cRO = k1lib.RunOnce(); cRO.done()
        for line in it:
            if col != None: line = list(line); elem = line[col]
            else: elem = line
            if self.pattern.search(elem): # new section
                self.sectionIdx += 1; counter = self.after+1; cRO.revert()
                if self.sectionIdx > self.N: return
                yield from queue; queue.clear(); yield line
            elif tillPattern is not None and tillPattern.search(elem) and counter == inf: # closing section
                counter = self.tillAfter + 1; cRO.revert(); yield line
            if counter == 0:
                queue.append(line) # saves recent past lines
            elif counter > 0: # yielding "after" section
                if cRO.done(): yield line
                counter -= 1

In [18]:
# joined, normal
assert "abcde12d34" | grep("d") | cli.deref() == ['d', 'd']
assert "abcde12d34" | grep("d", 1) | cli.deref() == ['c', 'd', '2', 'd']
assert "abcde12d34" | grep("d", 1, N=1) | cli.deref() == ['c', 'd']
assert "0123456789" | grep("4", after=1e9, N=1) | cli.deref() == ['4', '5', '6', '7', '8', '9']
# joined, till
assert "abcde12d34" | grep("d", N=1).till("e") | cli.deref() == ['d', 'e']
assert "abcde12d34" | grep("d").till("e") | cli.deref() == ['d', 'e', 'd', '3', '4']
assert ["abcde12d34", "abcde12d34"] | grep("d", sep=True).till("e").all() | cli.deref() == [[['d', 'e'], ['d', '3', '4']], [['d', 'e'], ['d', '3', '4']]]
assert range(10) | cli.join("") | grep("5").till("8") | cli.deref() == ['5', '6', '7', '8']
assert range(10) | cli.join("") | grep("5", N=1).till("8") | cli.deref() == ['5', '6', '7', '8']
assert "0123145" | grep("1", N=1).till("1") | cli.deref() == ['1', '2', '3']
assert "0123145" | grep("1", N=1).till() | cli.deref() == ['1', '2', '3']
assert "0123145" | grep("1", after=1e9, N=1) | cli.deref() == ['1', '2', '3']
# separated
assert "abcde12d34" | grep("d", 1, sep=True) | cli.deref() == [['c', 'd'], ['2', 'd']]
assert "abcde12d34" | grep("d", 1, N=1, sep=True) | cli.deref() == [['c', 'd']]
assert "0123145" | grep("1", sep=True).till() | cli.deref() == [['1', '2', '3'], ['1', '4', '5']]
assert "0123145" | grep("1", 4, 2, sep=True) | cli.deref() == [['0', '1', '2', '3'], ['1', '4', '5']]
assert "0123145" | grep("1", 2, 1, sep=True) | cli.deref() == [['0', '1', '2'], ['3', '1', '4']]
assert [['/reset', 2902], ['/users', 6], ['/users', 11], ['/user/2', 4],
 ['/user/2', 5], ['/properties', 3], ['/properties', 8], ['/user/2', 4],
 ['/users', 4], ['/user/2', 6], ['/properties', 4], ['/properties', 6]]\
| grep("/user/\d+", col=0) | cli.deref() == [['/user/2', 4], ['/user/2', 5], ['/user/2', 4], ['/user/2', 6]]

In [4]:
#export
class grepTemplate(BaseCli):
    def __init__(self, pattern:str, template:str):
        """Searches over all lines, pick out the match, and expands
it to the templateand yields"""
        super().__init__()
        self.pattern = re.compile(pattern); self.template = template
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it)
        for line in it:
            matchObj = self.pattern.search(line)
            if matchObj is None: continue
            yield matchObj.expand(self.template)

In [24]:
!../../export.py cli/grep

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.16a1
Uninstalling k1lib-0.16a1:
  Successfully uninstalled k1lib-0.16a1
running install
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/torch/lib/python3.8/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.e